In [1]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
genres = ['blues', 'classical', 'country','disco','hiphop','jazz','metal','pop','reggae','rock']
frame_length = 0.025
frame_stride = 0.010

# 화음 판별 추출 array 생성
maj_template = np.array([1,0,0, 0,1,0, 0,1,0, 0,0,0])
min_template = np.array([1,0,0, 1,0,0, 0,1,0, 0,0,0])
N_template   = np.array([1,1,1, 1,1,1, 1,1,1, 1,1,1.]) / 4.
# Generate the weighting matrix that maps chroma to labels
weights = np.zeros((25, 12), dtype=float)
labels = ['C:maj', 'C#:maj', 'D:maj', 'D#:maj', 'E:maj', 'F:maj',
          'F#:maj', 'G:maj', 'G#:maj', 'A:maj', 'A#:maj', 'B:maj',
          'C:min', 'C#:min', 'D:min', 'D#:min', 'E:min', 'F:min',
          'F#:min', 'G:min', 'G#:min', 'A:min', 'A#:min', 'B:min',
          'N']
for c in range(12):
    weights[c, :] = np.roll(maj_template, c) # c:maj
    weights[c + 12, :] = np.roll(min_template, c)  # c:min
weights[-1] = N_template  # the last row is the no-chord class
# Make a self-loop transition matrix over 25 states
trans = librosa.sequence.transition_loop(25, 0.9)


song_name = list(np.full(1000,''))
tempos = np.full(1000,0)
chords_1 = list(np.full(1000,''))
chords_ratio_1 = np.full(1000,0.00)
chords_2 = list(np.full(1000,''))
chords_ratio_2 = np.full(1000,0.00)
for g in range(10):
    for i in range(100):
        ii = ''
        if i<10:
            ii = '0'+str(i)
        else:
            ii = str(i)
        y, sr = librosa.load(f'./genres/{genres[g]}/{genres[g]}.000{ii}.wav', sr=22050)
        
        song_name[g*100 + i] =f'{genres[g]}.000{ii}.wav'
        print(f'{genres[g]}.000{ii}.wav')
        
        # 음원의 bpm 추출
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        tempos[g*100 + i] = tempo
        try:
            # 음원의 화음 추출
            y = librosa.effects.harmonic(y, margin=4)
            chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
            # Map chroma (observations) to class (state) likelihoods
            probs = np.exp(weights.dot(chroma))  # P[class | chroma] ~= exp(template' chroma)
            probs /= probs.sum(axis=0, keepdims=True)  # probabilities must sum to 1 in each column
            # Compute independent frame-wise estimates
            chords_ind = np.argmax(probs, axis=0)
            # And viterbi estimates
            chords_vit = librosa.sequence.viterbi_discriminative(probs, trans)
            
            # 화음 추출
            length = len(chords_vit)
            series = pd.Series(chords_vit)
            
            dictionary = dict(series.value_counts())
            chord_rank = list(dictionary.keys())
            chord_value = list(dictionary.values())
            chord_rank_1 = chord_rank[0]
            chord_rank_2 = chord_rank[1]
            chords_1[g*100+i] = labels[chord_rank_1]
            chords_2[g*100+i] = labels[chord_rank_2]
            chords_ratio_1[g*100+i] = round(chord_value[0]/length,3)
            chords_ratio_2[g*100+i] = round(chord_value[1]/length,3)
            print(chords_1[g*100+i],chords_ratio_1[g*100+i],chords_2[g*100+i],chords_ratio_2[g*100+i])
        except:
            continue

blues.00000.wav
G:maj 0.166 C:maj 0.143
blues.00001.wav
C:maj 0.572 G:min 0.278
blues.00002.wav
E:min 0.387 E:maj 0.292
blues.00003.wav
E:min 0.425 E:maj 0.262
blues.00004.wav
A#:min 0.793 A#:maj 0.07
blues.00005.wav
G:maj 0.599 G:min 0.143
blues.00006.wav
F:maj 0.739 F:min 0.139
blues.00007.wav
A#:maj 0.7 A#:min 0.183
blues.00008.wav
E:min 0.311 B:min 0.282
blues.00009.wav
G:maj 0.599 G:min 0.219
blues.00010.wav
F:maj 0.271 D#:maj 0.204
blues.00011.wav
E:min 0.492 E:maj 0.353
blues.00012.wav
B:min 0.667 E:maj 0.125
blues.00013.wav
B:min 0.528 G#:min 0.228
blues.00014.wav
B:min 0.541 A#:maj 0.278
blues.00015.wav
E:maj 0.295 B:min 0.288
blues.00016.wav
A:maj 0.605 C#:min 0.101
blues.00017.wav
A:min 0.358 F#:min 0.166
blues.00018.wav
B:min 0.159 D:maj 0.128
blues.00019.wav
A:min 0.609 C#:min 0.265
blues.00020.wav
E:maj 0.732 C:maj 0.146
blues.00021.wav
F#:min 0.248 D:maj 0.214
blues.00022.wav
F#:min 0.367 F#:maj 0.181
blues.00023.wav
A:maj 0.711 A:min 0.093
blues.00024.wav
A#:min 0.705 G

G:min 0.428 D:maj 0.222
classical.00095.wav
A#:maj 0.367 C:min 0.202
classical.00096.wav
D:maj 0.398 D:min 0.225
classical.00097.wav
A:min 0.357 A:maj 0.309
classical.00098.wav
F:maj 0.77 C:maj 0.114
classical.00099.wav
A#:min 0.221 C:min 0.214
country.00000.wav
F#:min 0.413 B:maj 0.161
country.00001.wav
B:maj 0.302 E:min 0.22
country.00002.wav
A:maj 0.648 A:min 0.079
country.00003.wav
G:maj 0.565 D:maj 0.151
country.00004.wav
A:min 0.323 E:maj 0.313
country.00005.wav
G:maj 0.403 C:maj 0.315
country.00006.wav
A#:maj 0.438 D#:maj 0.164
country.00007.wav
G#:maj 0.488 A#:min 0.259
country.00008.wav
E:maj 0.398 A:maj 0.252
country.00009.wav
G:min 0.657 C:maj 0.098
country.00010.wav
C:maj 0.436 F:maj 0.219
country.00011.wav
G:maj 0.469 D:maj 0.242
country.00012.wav
C:maj 0.434 F:maj 0.265
country.00013.wav
A:maj 0.38 D:maj 0.258
country.00014.wav
A:maj 0.326 D:maj 0.278
country.00015.wav
G#:maj 0.393 D#:maj 0.236
country.00016.wav
F:maj 0.355 A#:maj 0.237
country.00017.wav
D:maj 0.369 A:maj

F:maj 0.489 C:maj 0.189
disco.00094.wav
C:maj 0.589 A#:min 0.229
disco.00095.wav
D:min 0.345 G:min 0.258
disco.00096.wav
G:maj 0.383 C:maj 0.254
disco.00097.wav
disco.00098.wav
B:min 0.347 B:maj 0.229
disco.00099.wav
B:min 0.347 B:maj 0.229
hiphop.00000.wav
D:min 0.289 E:min 0.228
hiphop.00001.wav
hiphop.00002.wav
G:min 0.302 A#:maj 0.275
hiphop.00003.wav
A#:min 0.422 B:maj 0.22
hiphop.00004.wav
D:min 0.231 E:maj 0.203
hiphop.00005.wav
C:maj 0.693 E:min 0.118
hiphop.00006.wav
D#:min 0.34 C#:maj 0.285
hiphop.00007.wav
hiphop.00008.wav
E:min 0.191 F:maj 0.177
hiphop.00009.wav
E:min 0.411 A#:maj 0.169
hiphop.00010.wav
E:maj 0.218 A#:min 0.217
hiphop.00011.wav
A#:maj 0.345 D:min 0.16
hiphop.00012.wav
D#:maj 0.687 G#:maj 0.279
hiphop.00013.wav
G#:min 0.374 E:min 0.257
hiphop.00014.wav
G#:maj 0.845 C#:maj 0.056
hiphop.00015.wav
C:maj 0.59 G#:min 0.135
hiphop.00016.wav
G:maj 0.344 A:min 0.271
hiphop.00017.wav
F:min 0.476 C#:min 0.186
hiphop.00018.wav
A#:maj 0.585 C#:min 0.113
hiphop.00019.wav

G:min 0.186 G:maj 0.158
metal.00001.wav
F#:maj 0.234 B:maj 0.202
metal.00002.wav
F#:min 0.508 E:min 0.121
metal.00003.wav
A:min 0.663 F:min 0.091
metal.00004.wav
G:min 0.28 D:maj 0.165
metal.00005.wav
D:maj 0.434 E:maj 0.162
metal.00006.wav
A#:min 0.35 C#:maj 0.177
metal.00007.wav
F#:min 0.275 E:min 0.206
metal.00008.wav
E:min 0.86 G:maj 0.114
metal.00009.wav
E:maj 0.386 G:maj 0.198
metal.00010.wav
F#:min 0.197 E:min 0.187
metal.00011.wav
G#:min 0.294 G#:maj 0.266
metal.00012.wav
A:min 0.619 F:maj 0.133
metal.00013.wav
G:min 0.278 F:maj 0.176
metal.00014.wav
B:min 0.336 E:min 0.324
metal.00015.wav
G#:min 0.23 E:min 0.201
metal.00016.wav
E:maj 0.439 C#:maj 0.357
metal.00017.wav
D#:min 0.579 E:min 0.28
metal.00018.wav
C#:min 0.84 C#:maj 0.063
metal.00019.wav
D#:min 0.65 E:maj 0.143
metal.00020.wav
D:maj 0.275 C#:maj 0.245
metal.00021.wav
E:maj 0.241 C:maj 0.202
metal.00022.wav
A:maj 0.927 D:maj 0.037
metal.00023.wav
C:min 0.456 A:min 0.152
metal.00024.wav
E:min 0.337 B:min 0.328
metal.00

G:min 0.411 D:min 0.154
reggae.00009.wav
G#:maj 0.424 G#:min 0.207
reggae.00010.wav
G:maj 0.291 C:maj 0.181
reggae.00011.wav
D:maj 0.276 C:maj 0.268
reggae.00012.wav
A:min 0.663 E:maj 0.114
reggae.00013.wav
G:maj 0.183 E:min 0.181
reggae.00014.wav
D:maj 0.307 E:maj 0.222
reggae.00015.wav
A#:maj 0.585 D#:maj 0.225
reggae.00016.wav
G:maj 0.585 D:maj 0.111
reggae.00017.wav
F#:min 0.449 B:maj 0.178
reggae.00018.wav
A:min 0.344 D:min 0.338
reggae.00019.wav
C#:maj 0.226 D#:min 0.221
reggae.00020.wav
G:maj 0.474 E:min 0.238
reggae.00021.wav
G:maj 0.287 D:maj 0.257
reggae.00022.wav
C#:min 0.194 A:maj 0.176
reggae.00023.wav
G:min 0.362 D:min 0.177
reggae.00024.wav
D#:maj 0.35 A#:maj 0.247
reggae.00025.wav
C:maj 0.261 D:min 0.259
reggae.00026.wav
B:min 0.323 E:min 0.261
reggae.00027.wav
E:min 0.483 F:maj 0.365
reggae.00028.wav
E:maj 0.316 C#:maj 0.242
reggae.00029.wav
E:min 0.465 B:min 0.338
reggae.00030.wav
C#:min 0.872 F#:maj 0.037
reggae.00031.wav
C:maj 0.641 G:maj 0.246
reggae.00032.wav
D:mi

In [31]:
music_data = pd.DataFrame(np.zeros((1000,6)),columns = ['wav_name','tempos','chord_rank_1','chord_rank_1_ratio','chord_rank_2','chord_rank_2_ratio'])
music_data['wav_name'] = song_name
music_data['tempos'] = tempos
music_data['chord_rank_1'] = chords_1
music_data['chord_rank_1_ratio'] = chords_ratio_1
music_data['chord_rank_2'] = chords_2
music_data['chord_rank_2_ratio'] = chords_ratio_2

In [32]:
music_data

,wav_name,tempos,chord_rank_1,chord_rank_1_ratio,chord_rank_2,chord_rank_2_ratio
0,blues.00000.wav,123,G:maj,0.166,C:maj,0.143
1,blues.00001.wav,67,C:maj,0.572,G:min,0.278
2,blues.00002.wav,161,E:min,0.387,E:maj,0.292
3,blues.00003.wav,63,E:min,0.425,E:maj,0.262
4,blues.00004.wav,135,A#:min,0.793,A#:maj,0.070
...,...,...,...,...,...,...
995,rock.00095.wav,103,D:maj,0.590,G:maj,0.271
996,rock.00096.wav,117,A:maj,0.542,C#:min,0.201
997,rock.00097.wav,129,D:maj,0.239,G:maj,0.173
998,rock.00098.wav,73,C:maj,0.337,F:maj,0.312
